<a href="https://colab.research.google.com/github/Omen4Dead/collaboratory/blob/main/%22%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_0_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Импорт необходимых библиотек

import pandas as pd
import sqlite3
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib
import requests
import numpy as np

In [2]:
# Создайте базу данных sqlite, можно использовать sqlalchemy или pandas, или любой другой инструмент
# Задайте почту на которую будет отправлен отчет (создайте свою по примеру ранее, или воспользуйтесь готовой)
# "stepikairflowcourse", "123456aA-"
CONN = sqlite3.connect('curency.db')
MAIL = 'airflowcourcenick'
PASSWORD = 'GUaGP123qweasd!'

EMAIL = MAIL+'@yandex.ru'
HOST = 'smtp.yandex.ru'
CRED=(MAIL, PASSWORD)

In [3]:
# Напишите функцию которая принимает на вход дату
# и выгружает курс валюты EUR к USD за этот день
# Функция должна вернуть список словарей или можно использовать pandas.read_csv()

# Для этого воспользуйтесь https://exchangerate.host/#/
# и методом Timeseries Endpoint! Строго им. Где start_date = end_date = date
# Используйте параметры start_date, end_date, base=EUR, symbols=USD, format=csv

def extract_currency(date):
    url = f'https://api.exchangerate.host/timeseries?start_date={date}&end_date={date}&base=EUR&symbols=USD&format=csv'
    # response = requests.get(url)
    df = pd.read_csv(url)
    return df

In [4]:
# Напишите функцию которая скачивает данные с гитхаба
# Функция принимает на вход дату
# Функция должна вернуть список словарей или можно использовать pandas.read_csv()

# ссылка на гитхаб с файлами - https://github.com/dm-novikov/stepik_airflow_course/tree/main/data_new

def extract_data(date):
    url = f'https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/{date}.csv'
    df = pd.read_csv(url)
    return df

In [5]:
# Напишите функцию которая принимает на вход список словарей
# и загружает в табличку в sqlite, можно использоват pandas.to_sql()

def insert_to_db(data, table_name, conn=CONN):
  data['insert_time'] = pd.to_datetime('now')
  data.to_sql(table_name, conn, if_exists='replace', index=False)

In [6]:
#@title Текст заголовка по умолчанию
# Напишите функцию которая выполнит SQL запрос к базу данных
# Здесь удобно использовать sqlalchemy или https://pythonru.com/biblioteki/vvedenie-v-sqlite-python
# Если выполняется select ... то функция должна возвращать результат (не None)
# Если используется внутренняя операция например truncate table, insert into select ... то возвращает None
# data = sql_query(sql="select * from table") -> [(1, 2, 3), (4, 5, 6) ...]
# sql_query("truncate table") -> None

def sql_query(sql, conn=CONN):
    cursor = conn.cursor()
    res = cursor.execute(sql).fetchall()
    conn.commit()
    return res

In [7]:
def html_pretty(df):
    """ Pretty html dataframe
    """
    return """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(df.to_html())
    # """.format(df.to_html())

In [8]:
# Напишите функцию которая отправит полученный отчет на почту
# Для этого можете воспользоваться кодом который был написан ранее (на уроке)
# обернув его в вашу функцию

def send_report(data, email, host=HOST, port=587, to=EMAIL, From=EMAIL, cred=CRED):
    msg = MIMEMultipart()
    part = MIMEText(html_pretty(data), 'html')
    msg.attach(part)

    server = smtplib.SMTP(host, port)
    server.starttls()
    server.login(cred[0], cred[1])
    server.sendmail(From, to, msg.as_string())
    server.quit()

In [9]:
# Запустите ваш код в функции main

# Напишите генерацию дат, так чтобы у вас получился список
# 2021-01-02, 2021-01-03 ... etc
# Нужны даты с 2021-01-01 по 2021-01-04

dates_list = ['2021-01-0'+str(x) for x in range(1, 5)]

def main(date, email, conn):
  
  # Выгружаем данные по валютам и из источника
  currency = extract_currency(date)
  data = extract_data(date)

  # Создайте необходимые таблицы если это нужно
  # 'currency', 'data' и 'join_data' (таблица с объединенными данными)
  sql_query('''create table if not exists currency(date        date, 
                                                   code        text, 
                                                   rate        real, 
                                                   base        text, 
                                                   start_date  date, 
                                                   end_date  date, 
                                                   insert_time datetime)''')
  
  sql_query('''create table if not exists data(currency    text, 
                                               value       integer, 
                                               date        date, 
                                               insert_time datetime)''')
  
  sql_query('''create table if not exists join_data(currency    text, 
                                                    value       integer, 
                                                    date        date, 
                                                    rate        real, 
                                                    insert_time datetime default current_timestamp)''')

  # Вставляем данные в БД
  # У вас долнжо получиться 2 таблицы CURRENCY, DATA
  insert_to_db(currency, 'currency')
  insert_to_db(data, 'data')

  # Объединение данных в отдельную таблицу JOIN_DATA
  # CURRENCY, DATA объединить через JOIN по дате и валюте
  # Используйте insert into select ... inner join ...
  # И соответственно создать таблицу для этого

  sql_query(f"delete from join_data where date=date('{date}')")

  sql_query('''insert into join_data 
               (currency, value, date, rate)
               select d.currency, 
                      d.value, 
                      d.date, 
                      c.rate 
                 from data d
                inner join currency c on c.date = d.date
                                     and c.base = d.currency''')

  # Очистка временных таблиц CURRENCY, DATA
  # Это те таблицы куда произошла вставка данных
  # Используйте truncate
  sql_query('delete from currency')
  sql_query('delete from data')

  # Получение данных из таблицы JOIN_DATA за конкретный день
  # Используйте date при формировании запроса select * from ... where date ='...'
  report = sql_query(f"""select * 
                           from join_data 
                          where date=date('{date}')
                      """)
  
  dt = np.array(report)
  df = pd.DataFrame(dt, columns=['currency', 'value', 'date', 'rate', 'insert_time'])

  # Отправка данных на почту
  # Передайте данные в любом виде и отправьте на почту
  send_report(df, email)


# Пройдемся по списку дат и выполним скрипт
for date in dates_list:
  main(date, EMAIL, CONN)

In [10]:
# чтобы првоерить решение можете обратиться к вашей базе данных таким образом
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql sqlite://///content/curency.db
%sql select * from join_data

 * sqlite://///content/curency.db


,currency,value,date,rate,insert_time
0,EUR,38,2021-01-01,"1,217582",2022-03-26 19:50:21
1,EUR,65,2021-01-01,"1,217582",2022-03-26 19:50:21
2,EUR,74,2021-01-01,"1,217582",2022-03-26 19:50:21
3,EUR,42,2021-01-01,"1,217582",2022-03-26 19:50:21
4,EUR,23,2021-01-01,"1,217582",2022-03-26 19:50:21
5,EUR,48,2021-01-01,"1,217582",2022-03-26 19:50:21
6,EUR,86,2021-01-01,"1,217582",2022-03-26 19:50:21
7,EUR,74,2021-01-01,"1,217582",2022-03-26 19:50:21
8,EUR,24,2021-01-01,"1,217582",2022-03-26 19:50:21
9,EUR,27,2021-01-02,"1,2135",2022-03-26 19:50:24
